In [5]:
##IMPORT ##

import pandas as pd
pd.set_option('display.max_columns', None)
import math 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error
from sklearn.feature_selection import RFE
from sklearn.model_selection import HalvingGridSearchCV
import xgboost as xgb



## Explication Notebook : 

<p>Ce notebook est une explication de notre raisonnement pour obtenir nos résultats, certain code ne tourne plus correctement et d'autre sont très long à faire tourner. Le résultat final de notre expérience est dans le pipeline Kedro</p>


In [2]:
%reload_kedro
df_champs = catalog.load('df_champs_elysee_days_meteo_bank')
df_convention = catalog.load('df_convention_days_meteo_bank')
df_saint_peres = catalog.load('df_peres_days_meteo_bank')

[12/11/22 18:03:25] INFO     No path argument was provided. Using: C:\Users\Erwan                    __init__.py:56
                             BOULLIER\Desktop\Datathon\BCG_Hackaton\datathon-bcg-gamma                             

[12/11/22 18:03:25] INFO     Kedro project Datathon BCG Gamma                                        __init__.py:77

                    INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines' __init__.py:78

                    INFO     Registered line magic 'run_viz'                                         __init__.py:84

                    INFO     Loading data from 'df_champs_elysee_days_meteo_bank'               data_catalog.py:343
                             (ParquetDataSet)...                                                                   

                    INFO     Loading data from 'df_convention_days_meteo_bank'                  data_catalog.py:343
                             (ParquetDataSet)...                                                                   

[12/11/22 18:03:26] INFO     Loading data from 'df_peres_days_meteo_bank' (ParquetDataSet)...   data_catalog.py:343

## Data processing :
<p>Cette partie est consacrée au traitement des données. Nous effectuons deux traitements, le premier est le fênetrage (i.e l'ajout des données passées à l'input) et ensuite nous utilisons un pipeline Sklearn de normalisation des imputs</p>

In [ ]:
##Fenetrage simple
def series_to_supervised(df, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""

	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df[["Taux d'occupation",'Débit horaire']].shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(2)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df[["Taux d'occupation",'Débit horaire']].shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(2)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(2)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	df = df.merge(agg, left_index=True, right_index=True)
	# drop rows with NaN values
	if dropnan:
		df.dropna(inplace=True)
	return df


In [ ]:
#Fenetrage en utlisant la différence entre la valeur à T et T-1
def series_to_supervised_dif_2(df, n_in=1, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a list or NumPy array.
        n_in: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df[["Taux d'occupation",'Débit horaire']].shift(i) - df[["Taux d'occupation",'Débit horaire']].shift(i+1))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(2)]
    # forecast sequence (t, t+1, ... t+n)
    cols.append(df[["Taux d'occupation",'Débit horaire']])
    names += ['var_r1(t-1)', 'var_r2(t-1)']
    for i in range(0, n_out):
        cols.append(df[["Taux d'occupation",'Débit horaire']].shift(-i) - df[["Taux d'occupation",'Débit horaire']].shift(-1-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(2)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(2)]

    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    df = df.merge(agg, left_index=True, right_index=True)
    # drop rows with NaN values
    if dropnan:
        df.dropna(inplace=True)
    return df

In [ ]:
## Pipeline de normalization

def create_pipeline(num_col, min_max_col):
    ##Pipelines Scikit Learn
    ct_scaler = ColumnTransformer(
        transformers=[
            ("scaler", StandardScaler(), num_col),
            ("scaler_minmax", MinMaxScaler(), min_max_col),
        ]
    )

    pipeline_master = Pipeline([
        ("preprocessor", ct_scaler),
        ])
    
    return pipeline_master

In [4]:
## Normalisaton des outputs 
def normalize(data):
    return (data - data.mean())/data.std()

def unormalize(data, previous):
    return (data*previous.std()) + previous.mean()

In [ ]:
## Normalisation des outputs pour plusieurs predictions
def normalize_n(data):
    return (data - data.mean().mean())/data.std().mean()

def unormalize_n(data, previous):
    return (data*previous.std().mean()) + previous.mean().mean()

## Prédiction à T sans données Hexogènes

In [ ]:
df_time_only = catalog.load('champs_elysee_with_days_na_filled')
df_time_only = df_time_only.drop(columns=['Libelle','Date et heure de comptage','date','vacances'])
df_time_only = pd.get_dummies(df_time_only, columns=['year'], prefix='year')
df_time_only = pd.get_dummies(df_time_only, columns=['hour'], prefix='hour')
df_time_only = pd.get_dummies(df_time_only, columns=['day'], prefix='day')
df_time_only = pd.get_dummies(df_time_only, columns=['day_of_week'], prefix='wday')
df_time_only = pd.get_dummies(df_time_only, columns=['month'], prefix='month')


df_time_only_preprocessd = series_to_supervised(df_time_only, 48, 1) #Using 48 past values
df_time_only_preprocessd = df_time_only_preprocessd.drop(columns=["Taux d'occupation","Débit horaire"]) 
y_taux_t_time_only = df_time_only_preprocessd['var1(t)']
y_debit_t_time_only = df_time_only_preprocessd['var2(t)']
X_time_only = df_time_only_preprocessd.drop(columns=['var1(t)','var2(t)'])

##Split
offset = int(X_time_only.shape[0] * 0.9)
X_train_time_only, y_taux_train_time_only, y_debit_train_time_only = X_time_only[:offset], y_taux_t_time_only[:offset], y_debit_t_time_only[:offset]
X_test_time_only, y_taux_test_time_only, y_debit_test_time_only = X_time_only[offset:], y_taux_t_time_only[offset:], y_debit_t_time_only[offset:]


#Models
model_taux_time_only = xgb.XGBRegressor()
model_taux_time_only = model_taux_time_only.fit(X_train_time_only,y_taux_train_time_only) 

model_debit_time_only = xgb.XGBRegressor()
model_debit_time_only = model_debit_time_only.fit(X_train_time_only,y_debit_train_time_only) 


#Testing
y_pred_taux_time_only_t = model_taux_time_only.predict(X_test_time_only)
RMSE_taux_time_only = mean_squared_error(y_taux_test_time_only, y_pred_taux_time_only_t)


y_pred_debit_time_only_t = model_debit_time_only.predict(X_test_time_only)
RMSE_debit_time_only = mean_squared_error(y_debit_test_time_only, y_pred_debit_time_only_t)

print(RMSE_taux_time_only**0.5,RMSE_debit_time_only**0.5)

fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(18, 5))
n_taux_time_only = len(y_taux_test_time_only)
t_taux_time_only = np.arange(n_taux_time_only)
# plt.figure("Taux d'occupation")
ax[0,0].plot(t_taux_time_only,y_taux_test_time_only, 'r')
ax[0,0].plot(t_taux_time_only,y_pred_taux_time_only_t, 'b')
ax[0,0].title.set_text("Taux d'occupation")

n_debit_time_only= len(y_pred_debit_time_only_t)
t_debit_time_only = np.arange(n_debit_time_only)
# plt.figure("Débit horaire")
ax[0,1].plot(t_debit_time_only,y_debit_test_time_only, 'r')
ax[0,1].plot(t_debit_time_only,y_pred_debit_time_only_t, 'b')
ax[0,1].title.set_text("Débit horaire")

t_zoom = np.arange(100)
ax[1,0].plot(t_zoom,y_taux_test_time_only[:100], 'r')
ax[1,0].plot(t_zoom,y_pred_taux_time_only_t[:100], 'b')

ax[1,1].plot(t_zoom,y_debit_test_time_only[:100], 'r')
ax[1,1].plot(t_zoom,y_pred_debit_time_only_t[:100], 'b')
fig.legend(['Real Data','Predicted'])
plt.show()


## Prediction à t en utilisant des données Hexogènes 

In [ ]:
df_preprocessed_champs = series_to_supervised(df_champs, 48, 1) ## Using 48 past data
df_preprocessed_champs = df_preprocessed_champs.drop(columns=["Taux d'occupation","Débit horaire"]) 
y_taux_t = df_preprocessed_champs['var1(t)']

y_debit_t = df_preprocessed_champs['var2(t)']

names_taux_drop = [('var2(t-%d)' % (i)) for i in range(1,48)]
names_debit_drop = [('var1(t-%d)' % (j)) for j in range(1,48)]
X_taux = df_preprocessed_champs.drop(columns=['var1(t)','var2(t)'] + names_taux_drop)
X_debit = df_preprocessed_champs.drop(columns=['var1(t)','var2(t)']+ names_debit_drop)
X_taux = X_taux.drop(columns=['Libelle','Date et heure de comptage','date'])
X_debit = X_debit.drop(columns=['Libelle','Date et heure de comptage','date'])

## Data split 
X_train_taux, X_test_taux,y_taux_train, y_taux_test = train_test_split(X_taux,y_taux_t, test_size=0.1)
X_train_debit, X_test_debit,y_debit_train, y_debit_test = train_test_split(X_debit,y_debit_t, test_size=0.2)

y_taux_train_norm = normalize(y_taux_train)

y_debit_train_norm = normalize(y_debit_train)

## Features type declaration
min_max_columns = ['vacances','est_ferie','year_2021','year_2022','hour_0','hour_1','hour_2','hour_3','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23','day_1','day_2','day_3','day_4','day_5','day_6','day_7','day_8','day_9','day_10','day_11','day_12','day_13','day_14','day_15','day_16','day_17','day_18','day_19','day_20','day_21','day_22','day_23','day_24','day_25','day_26','day_27','day_28','day_29','day_30','day_31','wday_Monday','wday_Tuesday','wday_Wednesday','wday_Thursday','wday_Friday','wday_Saturday','wday_Sunday','month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']
numerical_col_taux = [feature for feature in list(X_taux) if feature not in min_max_columns]
numerical_col_debit = [feature for feature in list(X_debit) if feature not in min_max_columns]

## Normalisation : 
pipeline_taux = create_pipeline(numerical_col_taux,min_max_columns)
X_train_taux_preprocessed = pipeline_taux.fit_transform(X_train_taux)
X_test_taux_preprocessed = pipeline_taux.transform(X_test_taux)


pipeline_debit = create_pipeline(numerical_col_debit,min_max_columns)
X_train_debit_preprocessed = pipeline_debit.fit_transform(X_train_debit)
X_test_debit_preprocessed = pipeline_debit.transform(X_test_debit)

##Modelisation : 

## Prediction of taux at t : 
model_taux_t = xgb.XGBRegressor()
model_taux_t = model_taux_t.fit(X_train_taux_preprocessed,y_taux_train_norm)

## Prediction of débit at t : 
model_debit_t = xgb.XGBRegressor()
model_debit_t = model_debit_t.fit(X_train_debit_preprocessed, y_debit_train_norm)


#Testing data 
y_pred_taux_t = model_taux_t.predict(X_test_taux_preprocessed)
y_pred_taux_t = unormalize(y_pred_taux_t, y_taux_train)
MSE_taux = mean_squared_error(y_taux_test, y_pred_taux_t)

y_pred_debit_t = model_debit_t.predict(X_test_debit_preprocessed)
y_pred_debit_t = unormalize(y_pred_debit_t, y_debit_train)
MSE_debit = mean_squared_error(y_debit_test,y_pred_debit_t)

print(MSE_taux**0.5,y_taux_test.std(),MSE_debit**0.5,y_debit_test.std())

#Plot 
fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(18, 5))
n_taux = len(y_taux_test)
t_taux = np.arange(n_taux)
# plt.figure("Taux d'occupation")
ax[0,0].plot(t_taux,y_taux_test, 'r')
ax[0,0].plot(t_taux,y_pred_taux_t, 'b')
ax[0,0].title.set_text("Taux d'occupation")
n_debit= len(y_debit_test)
t_debit = np.arange(n_debit)
# plt.figure("Débit horaire")
ax[0,1].plot(t_debit,y_debit_test, 'r')
ax[0,1].plot(t_debit,y_pred_debit_t, 'b')
ax[0,1].title.set_text("Débit horaire")

t_zoom = np.arange(100)
ax[1,0].plot(t_zoom,y_taux_test[:100], 'r')
ax[1,0].plot(t_zoom,y_pred_taux_t[:100], 'b')

ax[1,1].plot(t_zoom,y_debit_test[:100], 'r')
ax[1,1].plot(t_zoom,y_pred_debit_t[:100], 'b')
fig.legend(['Real Data','Predicted'])
plt.show()

## Prediction de K valeurs futures (with outputs noramlized)

In [ ]:
## To reload on Dej because error
# params = {'colsample_bytree': 0.3, 'max_depth': 8, 'min_child_weight': 5, 'n_estimators': 43}
df_preprocessed_champs_n = series_to_supervised(df_champs, 12, 24)
df_preprocessed_champs_n = df_preprocessed_champs_n.drop(columns=["Taux d'occupation","Débit horaire"]) 
names_taux = ['var1(t)']
names_taux += [('var%d(t+%d)' % (1, i)) for i in range(1,24)]
y_taux_n =  df_preprocessed_champs_n[names_taux]
names_debit = ['var2(t)']
names_debit += [('var%d(t+%d)' % (2, i)) for i in range(1,24)]
y_debit_n = df_preprocessed_champs_n[names_debit]
X_past = df_preprocessed_champs_n.drop(columns=names_debit+names_taux)
X_past = X_past.drop(columns=['Libelle','Date et heure de comptage','date'])

## Data split 
X_train_taux_n , X_test_taux_n ,y_taux_train_n , y_taux_test_n = train_test_split(X_past,y_taux_n, test_size=0.2)
X_train_debit_n, X_test_debit_n,y_debit_train_n, y_debit_test_n = train_test_split(X_past,y_debit_n, test_size=0.2)

y_taux_train_n_norm = normalize_n(y_taux_train_n)
y_debit_train_n_norm = normalize_n(y_debit_train_n)

## Features type declaration
min_max_columns_n = ['vacances','est_ferie','year_2021','year_2022','hour_0','hour_1','hour_2','hour_3','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23','day_1','day_2','day_3','day_4','day_5','day_6','day_7','day_8','day_9','day_10','day_11','day_12','day_13','day_14','day_15','day_16','day_17','day_18','day_19','day_20','day_21','day_22','day_23','day_24','day_25','day_26','day_27','day_28','day_29','day_30','day_31','wday_Monday','wday_Tuesday','wday_Wednesday','wday_Thursday','wday_Friday','wday_Saturday','wday_Sunday','month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']
numerical_col_n = [feature for feature in list(X_past) if feature not in min_max_columns_n]

#Normalization
pipeline_n = create_pipeline(numerical_col_n,min_max_columns_n)
X_train_taux_n_preprocessed = pipeline_n.fit_transform(X_train_taux_n)
X_test_taux_n_preprocessed = pipeline_n.transform(X_test_taux_n)
X_train_debit_n_preprocessed = pipeline_n.fit_transform(X_train_debit_n)
X_test_debit_n_preprocessed = pipeline_n.transform(X_test_debit_n)

#Models 

## Prediction of taux at t : 
print('Launch Model')
model_taux_n = xgb.XGBRegressor()
model_taux_n = model_taux_n.fit(X_train_taux_n_preprocessed,y_taux_train_n_norm)
print('ok') 
## Prediction of débit at t : 
model_debit_n = xgb.XGBRegressor()
model_debit_n = model_debit_n.fit(X_train_debit_n_preprocessed, y_debit_train_n_norm)
print('ok')


y_pred_taux_n = model_taux_n.predict(X_test_taux_n_preprocessed)
y_pred_taux_n = unormalize_n(y_pred_taux_n, y_taux_train_n)
MSE_taux_n = mean_squared_error(y_taux_test_n, y_pred_taux_n)


y_pred_debit_n = model_debit_n.predict(X_test_debit_n_preprocessed)
y_pred_debit_n = unormalize_n(y_pred_debit_n, y_debit_train_n)
MSE_debit_n = mean_squared_error(y_debit_test_n, y_pred_debit_n)

print(MSE_taux_n**0.5,MSE_debit_n**0.5)

fig, ax = plt.subplots(ncols=2, nrows=2, figsize=(18, 5))
n_taux_n = len(y_taux_test_n)
t_taux_n = np.arange(n_taux_n)
# plt.figure("Taux d'occupation")
ax[0,0].plot(t_taux_n,y_taux_test_n, 'r')
ax[0,0].plot(t_taux_n,y_pred_taux_n, 'b')
ax[0,0].title.set_text("Taux d'occupation")


n_debit_n= len(y_debit_test_n)
t_debit_n = np.arange(n_debit_n)
# plt.figure("Débit horaire")
ax[0,1].plot(t_debit_n,y_debit_test_n, 'r') 
ax[0,1].plot(t_debit_n,y_pred_debit_n, 'b')
ax[0,1].title.set_text("Débit horaire")

t_zoom = np.arange(len(y_taux_test_n.iloc[-1, :]))
ax[1,0].plot(t_zoom,y_taux_test_n.iloc[-1, :], 'r')
ax[1,0].plot(t_zoom,y_pred_taux_n[-1], 'b')

ax[1,1].plot(t_zoom,y_debit_test_n.iloc[0, :], 'r')
ax[1,1].plot(t_zoom,y_pred_debit_n[0], 'b')
ax[1,1].legend(['real','pred'])
# fig.legend(['Real Data','Predicted'])
plt.show()


## Feature Tunning : 
<p>Nous avons utiliser 2 type d'optimisation de feature tunning. Nous avons fait une RFE pour réduire le nombre de paramètres et un HalvingGridSearch pour toruver les paramètres optimaux de xgb.Regressor</p>

In [ ]:
## RFE
df_preprocessed_champs_n = series_to_supervised(df_champs, 12, 24)
df_preprocessed_champs_n = df_preprocessed_champs_n.drop(columns=["Taux d'occupation","Débit horaire"]) 
names_taux = ['var1(t)']
names_taux += [('var%d(t+%d)' % (1, i)) for i in range(1,24)]
y_taux_n =  df_preprocessed_champs_n[names_taux]
names_debit = ['var2(t)']
names_debit += [('var%d(t+%d)' % (2, i)) for i in range(1,24)]
y_debit_n = df_preprocessed_champs_n[names_debit]
X_past = df_preprocessed_champs_n.drop(columns=names_debit+names_taux)
X_past = X_past.drop(columns=['Libelle','Date et heure de comptage','date'])

X_train_taux_n , X_test_taux_n ,y_taux_train_n , y_taux_test_n = train_test_split(X_past,y_taux_n, test_size=0.2, random_state=1)
X_train_debit_n, X_test_debit_n,y_debit_train_n, y_debit_test_n = train_test_split(X_past,y_debit_n, test_size=0.2, random_state=1)

y_taux_train_n_norm = normalize_n(y_taux_train_n)
y_debit_train_n_norm = normalize_n(y_debit_train_n)

## Features type declaration
min_max_columns_n = ['vacances','est_ferie','year_2021','year_2022','hour_0','hour_1','hour_2','hour_3','hour_4','hour_5','hour_6','hour_7','hour_8','hour_9','hour_10','hour_11','hour_12','hour_13','hour_14','hour_15','hour_16','hour_17','hour_18','hour_19','hour_20','hour_21','hour_22','hour_23','day_1','day_2','day_3','day_4','day_5','day_6','day_7','day_8','day_9','day_10','day_11','day_12','day_13','day_14','day_15','day_16','day_17','day_18','day_19','day_20','day_21','day_22','day_23','day_24','day_25','day_26','day_27','day_28','day_29','day_30','day_31','wday_Monday','wday_Tuesday','wday_Wednesday','wday_Thursday','wday_Friday','wday_Saturday','wday_Sunday','month_1','month_2','month_3','month_4','month_5','month_6','month_7','month_8','month_9','month_10','month_11','month_12']
numerical_col_n = [feature for feature in list(X_past) if feature not in min_max_columns_n]

#Normalization
pipeline_n = create_pipeline(numerical_col_n,min_max_columns_n)
X_train_taux_n_preprocessed = pipeline_n.fit_transform(X_train_taux_n)
X_test_taux_n_preprocessed = pipeline_n.transform(X_test_taux_n)
X_train_debit_n_preprocessed = pipeline_n.fit_transform(X_train_debit_n)
X_test_debit_n_preprocessed = pipeline_n.transform(X_test_debit_n)

model_taux_n = xgb.XGBRegressor()
RFE_taux = RFE(estimator=model_taux_n, n_features_to_select=69, step=1)
RFE_taux.fit(X_past,y_taux_n)
## Prediction of débit at t : 
model_debit_n = xgb.XGBRegressor(X_past,y_debit_n)
RFE_debit = RFE(estimator=model_debit_n, n_features_to_select=69, step=1)

In [ ]:
## Grid Search 
gbm_params_grid = {
    'max_depth':[5,6,7,8,9],
    'min_child_weight': [4,5,6,7,8],
    'n_estimators': [i for i in range(30,51)], 
    'colsample_bytree':[0.1,0.2,0.3]
}
estim = xgb.XGBRegressor()
search = HalvingGridSearchCV(estim, gbm_params_grid,cv=3,scoring='neg_mean_squared_error', n_jobs=-1).fit(X_past,y_debit_n)


